# 2.1 Imports

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from requests.exceptions import ReadTimeout

import time
import difflib
import requests
import regex as re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2.2 Setup

### 2.2.1 Spotipy

In [3]:
keys = extensions = open('../keys/client_keys.txt','r').read().split('\n')
#tokens from spotify dev account to access api
client_id = keys[0].split('=')[1].strip()
client_secret = keys[1].split('=')[1].strip()

In [4]:
#authetification for spotpipy
auth_manager = SpotifyClientCredentials(client_id=client_id, 
                                        client_secret=client_secret)
#spotify search call
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=20)

### 2.2.2 Requests

Obtained from [this stackoverflow post](https://stackoverflow.com/questions/74936648/getting-object-of-type-none-is-not-subscriptable-error-while-using-spotify-api)

In [3]:
auth_url = 'https://accounts.spotify.com/api/token'
base_url = 'https://api.spotify.com/v1/'

#function to generate authentication token in case it expires
def getAuth():
    #post
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()

    # save the access token
    access_token = auth_response_data['access_token']

    #header for authentification
    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)
    }
    
    return headers

headers = getAuth()

## 2.3 Load Data

In [4]:
album_df = pd.read_csv('../data/movie_albums.csv',index_col=0)

In [5]:
album_df.head()

,movie_id,movie_name,movie_year,name,release_date,release_date_precision,id,artists,album_type,total_tracks,type,uri
0,0,Aladdin,1992,Aladdin (Original Motion Picture Soundtrack),1992-01-01,day,29EiOQJnxWlX5nVOWQpu3u,Various Artists,compilation,21,album,spotify:album:29EiOQJnxWlX5nVOWQpu3u
1,2,Annie,1999,Annie - Original Telefilm Soundtrack,1999-10-04,day,4rEVfFhY5aP2oYee7t8eqn,Charles Strouse,album,17,album,spotify:album:4rEVfFhY5aP2oYee7t8eqn
2,3,Babes in Toyland,1961,Babes in Toyland,1961-01-01,day,7CgMMSHPx6pXIi3CG8SpLt,Various Artists,compilation,16,album,spotify:album:7CgMMSHPx6pXIi3CG8SpLt
3,4,Beauty and the Beast,1991,Beauty and the Beast,1991-01-01,day,3O5p9VNddbwvqWTdYKEqV5,Various Artists,compilation,16,album,spotify:album:3O5p9VNddbwvqWTdYKEqV5
4,5,Beauty and the Beast,2017,Beauty and the Beast (Original Motion Picture ...,2017-03-10,day,3AhP76AusdcJ9sZ4XgUcEc,Various Artists,compilation,52,album,spotify:album:3AhP76AusdcJ9sZ4XgUcEc


We want to create a DataFrame made up of all the tracks for each album, to do so we look at `album_df['id']` to see what each Spotify album's ID looks like

In [7]:
id_list = list(album_df.id)

In [8]:
album_id = album_df.iloc[4].id
album_tracks = album_df.iloc[4].total_tracks

album_id,album_tracks

('3AhP76AusdcJ9sZ4XgUcEc', 52)

Spotify api call returns a maximum of 50 results, we see what is the most number of tracks to see how to handle this

In [9]:
album_df.total_tracks.max()

59

In [10]:
def getAllTracks(album_id):
    #50 is maximum limit
    r = requests.get(base_url+'albums/{idx}/tracks?limit=50'.format(idx=album_id),headers = headers)
    r = r.json()
    if album_tracks>50:
        #get remaining songs
        r2 = requests.get(base_url+'albums/{idx}/tracks?offset=50&limit=50'.format(idx=album_id),headers = headers)
        r2 = r2.json()

        #extend list of first 50 seconds with remaining number of songs
        r['items'] = r['items']+r2['items']
        
    return r['items']

all_tracks = getAllTracks(album_id)
for (i,track) in enumerate(all_tracks):
    print(i+1,track['name'])

1 How Does A Moment Last Forever
2 Beauty and the Beast
3 Evermore
4 Overture
5 Main Title: Prologue Pt. 1
6 Aria
7 Main Title: Prologue Pt. 2
8 Belle
9 How Does A Moment Last Forever (Music Box)
10 Belle (Reprise)
11 Gaston
12 Be Our Guest
13 Days In The Sun
14 Something There
15 How Does A Moment Last Forever (Montmartre)
16 Beauty and the Beast
17 Evermore
18 The Mob Song
19 Beauty and the Beast (Finale)
20 Aria - Demo
21 How Does A Moment Last Forever (Music Box) - Demo
22 Days In The Sun - Demo
23 How Does A Moment Last Forever (Montmartre) - Demo
24 Evermore - Demo
25 Main Title: Prologue
26 Belle Meets Gaston
27 Your Mother
28 The Laverie
29 Wolf Chase
30 Entering the Castle
31 A White Rose
32 The Beast
33 Meet the Staff
34 Home - Extended Mix
35 Madame De Garderobe
36 There's a Beast
37 A Petal Drops
38 A Bracing Cup of Tea
39 The West Wing
40 Wolves Attack Belle
41 The Library
42 Colonnade Chat
43 The Plague
44 Maurice Accuses Gaston
45 Beast Takes a Bath
46 The Dress
47 You M

In [11]:
pd.DataFrame(data=all_tracks).head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,is_local,name,preview_url,track_number,type,uri
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,217537,False,{'spotify': 'https://open.spotify.com/track/0C...,https://api.spotify.com/v1/tracks/0CJvw1j88DGb...,0CJvw1j88DGbvDhqmjJgc8,False,How Does A Moment Last Forever,None,1,track,spotify:track:0CJvw1j88DGbvDhqmjJgc8
1,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,227928,False,{'spotify': 'https://open.spotify.com/track/7w...,https://api.spotify.com/v1/tracks/7wMPhUSe6CZg...,7wMPhUSe6CZga1vOMpLTJP,False,Beauty and the Beast,None,2,track,spotify:track:7wMPhUSe6CZga1vOMpLTJP
2,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,189477,False,{'spotify': 'https://open.spotify.com/track/4t...,https://api.spotify.com/v1/tracks/4t90YkTzNWji...,4t90YkTzNWjis7HM2NqQLQ,False,Evermore,None,3,track,spotify:track:4t90YkTzNWjis7HM2NqQLQ
3,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,185560,False,{'spotify': 'https://open.spotify.com/track/1I...,https://api.spotify.com/v1/tracks/1IvMhKEYofTz...,1IvMhKEYofTzkDHXaOan1B,False,Overture,None,4,track,spotify:track:1IvMhKEYofTzkDHXaOan1B
4,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,42640,False,{'spotify': 'https://open.spotify.com/track/3v...,https://api.spotify.com/v1/tracks/3vYoq9YnTdpH...,3vYoq9YnTdpHDuWleiXai5,False,Main Title: Prologue Pt. 1,None,5,track,spotify:track:3vYoq9YnTdpHDuWleiXai5


In [12]:
pd.DataFrame(data=all_tracks).columns, len(pd.DataFrame(data=all_tracks).columns)

(Index(['artists', 'available_markets', 'disc_number', 'duration_ms',
        'explicit', 'external_urls', 'href', 'id', 'is_local', 'name',
        'preview_url', 'track_number', 'type', 'uri'],
       dtype='object'),
 14)

Unfortunately, we do not get song popularity by doing this, so we have to call Spotify API by track ID:

In [13]:
r = requests.get("https://api.spotify.com/v1/tracks/7wMPhUSe6CZga1vOMpLTJP",headers=headers)
r = r.json()

r.keys(),len(r.keys())

(dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri']),
 17)

In [14]:
def getAllTrackIds(album_id):
    #50 is maximum limit
    r = requests.get(base_url+'albums/{idx}/tracks?limit=50'.format(idx=album_id),headers = headers)
    r = r.json()
    if album_tracks>50:
        #get remaining songs
        r2 = requests.get(base_url+'albums/{idx}/tracks?offset=50&limit=50'.format(idx=album_id),headers = headers)
        r2 = r2.json()

        #extend list of first 50 seconds with remaining number of songs
        r['items'] = r['items']+r2['items']
    
    track_ids = [track['id'] for track in r['items']]
    
    return track_ids

In [15]:
track_ids = getAllTrackIds('3AhP76AusdcJ9sZ4XgUcEc')
track_ids[:5]

['0CJvw1j88DGbvDhqmjJgc8',
 '7wMPhUSe6CZga1vOMpLTJP',
 '4t90YkTzNWjis7HM2NqQLQ',
 '1IvMhKEYofTzkDHXaOan1B',
 '3vYoq9YnTdpHDuWleiXai5']

In [16]:
def getTrackInfo(track_id):
    try:
        r = requests.get("http://api.spotify.com/v1/tracks/{track_id}".format(track_id=track_id),headers=headers)
        r = r.json()
    except ReadTimeout:    #spotify sometimes times out and we have to retry
        print('Spotify timed out... trying again...')
        #repeat as above
        r = requests.get("http://api.spotify.com/v1/tracks/{track_id}".format(track_id=track_id),headers=headers)
        r = r.json()
    
    return r

In [17]:
all_id_tuples = []

for album_id in id_list:
    track_ids = getAllTrackIds(album_id)
    
    album_track_tuple = [(album_id,track_id) for track_id in track_ids]
    
    all_id_tuples = all_id_tuples + album_track_tuple

In [19]:
all_id_tuples[:4]

[('29EiOQJnxWlX5nVOWQpu3u', '0CKmN3Wwk8W4zjU0pqq2cv'),
 ('29EiOQJnxWlX5nVOWQpu3u', '0dQKabsoT9ojccQgVhLmHN'),
 ('29EiOQJnxWlX5nVOWQpu3u', '4wN8Ov3kPZdkJ8XcYxYUGz'),
 ('29EiOQJnxWlX5nVOWQpu3u', '4Y8ffsPwnVRkFEUqxkQAyz')]

In [18]:
len(all_id_tuples)

1857

In [44]:
all_tracks = []

for (album_id, track_id) in all_id_tuples:
    track_dict = getTrackInfo(track_id)
    track_dict['album_id']=album_id
    all_tracks.append(track_dict)

In [45]:
len(all_tracks)

1857

In [46]:
tracks_df = pd.DataFrame(data=all_tracks)
tracks_df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album_id
0,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,79293,False,{'isrc': 'USWD10423197'},{'spotify': 'https://open.spotify.com/track/0C...,https://api.spotify.com/v1/tracks/0CKmN3Wwk8W4...,0CKmN3Wwk8W4zjU0pqq2cv,False,Arabian Nights,58,None,1,track,spotify:track:0CKmN3Wwk8W4zjU0pqq2cv,29EiOQJnxWlX5nVOWQpu3u
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,85186,False,{'isrc': 'USWD10423198'},{'spotify': 'https://open.spotify.com/track/0d...,https://api.spotify.com/v1/tracks/0dQKabsoT9oj...,0dQKabsoT9ojccQgVhLmHN,False,Legend of the Lamp,36,None,2,track,spotify:track:0dQKabsoT9ojccQgVhLmHN,29EiOQJnxWlX5nVOWQpu3u
2,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,142440,False,{'isrc': 'USWD10423200'},{'spotify': 'https://open.spotify.com/track/4w...,https://api.spotify.com/v1/tracks/4wN8Ov3kPZdk...,4wN8Ov3kPZdkJ8XcYxYUGz,False,One Jump Ahead,54,None,3,track,spotify:track:4wN8Ov3kPZdkJ8XcYxYUGz,29EiOQJnxWlX5nVOWQpu3u
3,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,112946,False,{'isrc': 'USWD10423201'},{'spotify': 'https://open.spotify.com/track/4Y...,https://api.spotify.com/v1/tracks/4Y8ffsPwnVRk...,4Y8ffsPwnVRkFEUqxkQAyz,False,Street Urchins,35,None,4,track,spotify:track:4Y8ffsPwnVRkFEUqxkQAyz,29EiOQJnxWlX5nVOWQpu3u
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,61360,False,{'isrc': 'USWD10423202'},{'spotify': 'https://open.spotify.com/track/6L...,https://api.spotify.com/v1/tracks/6LNdgw2RMWQL...,6LNdgw2RMWQLrlGUzcUO4z,False,One Jump Ahead (Reprise),40,None,5,track,spotify:track:6LNdgw2RMWQLrlGUzcUO4z,29EiOQJnxWlX5nVOWQpu3u


In [61]:
tracks_df.iloc[208].artists

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kHxkdiKCSnHMkhIgFBZaI'},
  'href': 'https://api.spotify.com/v1/artists/2kHxkdiKCSnHMkhIgFBZaI',
  'id': '2kHxkdiKCSnHMkhIgFBZaI',
  'name': 'Kristen Bell',
  'type': 'artist',
  'uri': 'spotify:artist:2kHxkdiKCSnHMkhIgFBZaI'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/6kHu5ICQRBZdsAHjAjQlmp'},
  'href': 'https://api.spotify.com/v1/artists/6kHu5ICQRBZdsAHjAjQlmp',
  'id': '6kHu5ICQRBZdsAHjAjQlmp',
  'name': 'Agatha Lee Monn',
  'type': 'artist',
  'uri': 'spotify:artist:6kHu5ICQRBZdsAHjAjQlmp'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5Kkhk21siwbW3BfZUwpykJ'},
  'href': 'https://api.spotify.com/v1/artists/5Kkhk21siwbW3BfZUwpykJ',
  'id': '5Kkhk21siwbW3BfZUwpykJ',
  'name': 'Katie Lopez',
  'type': 'artist',
  'uri': 'spotify:artist:5Kkhk21siwbW3BfZUwpykJ'}]

In [84]:
tracks_df = pd.DataFrame(data=all_tracks)
sample_rows = tracks_df.iloc[[207,208]].copy()
sample_rows['artists_id'] = sample_rows.artists.apply(lambda artists: [artist['id'] for artist in artists])
sample_rows.artists = sample_rows.artists.apply(lambda artists: [artist['name'] for artist in artists])
sample_rows[['name','artists','artists_id']]

,name,artists,artists_id
207,"Frozen Heart - From ""Frozen""/Soundtrack Version",[Cast - Frozen],[2OZsw4C2nujSgVUfpuOTWL]
208,"Do You Want to Build a Snowman? - From ""Frozen...","[Kristen Bell, Agatha Lee Monn, Katie Lopez]","[2kHxkdiKCSnHMkhIgFBZaI, 6kHu5ICQRBZdsAHjAjQlm..."


In [86]:
sample_rows.explode(['artists','artists_id'])[['name','artists','artists_id']]

,name,artists,artists_id
207,"Frozen Heart - From ""Frozen""/Soundtrack Version",Cast - Frozen,2OZsw4C2nujSgVUfpuOTWL
208,"Do You Want to Build a Snowman? - From ""Frozen...",Kristen Bell,2kHxkdiKCSnHMkhIgFBZaI
208,"Do You Want to Build a Snowman? - From ""Frozen...",Agatha Lee Monn,6kHu5ICQRBZdsAHjAjQlmp
208,"Do You Want to Build a Snowman? - From ""Frozen...",Katie Lopez,5Kkhk21siwbW3BfZUwpykJ


In [29]:
tracks_df.columns

Index(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local',
       'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri',
       'album_id'],
      dtype='object')

In [31]:
# tracks_df.to_csv('../data/raw_tracks.csv')

In [87]:
tracks_df = pd.DataFrame(data=all_tracks)

tracks_df['artists_id'] = tracks_df['artists'].apply(lambda artists: 
                                                     [artist['id'] for artist in artists 
                                                      if 'Disney' not in artist['name']])
tracks_df['artists'] = tracks_df['artists'].apply(lambda artists: 
                                                  [artist['name'] for artist in artists 
                                                   if 'Disney' not in artist['name']])
unwanted_cols = ['album','available_markets','external_ids','external_urls','href','preview_url']
tracks_df = tracks_df.drop(unwanted_cols,axis=1)
tracks_df = tracks_df.explode(['artists','artists_id'])
tracks_df.head(10)

,artists,disc_number,duration_ms,explicit,id,is_local,name,popularity,track_number,type,uri,album_id,artists_id
0,Bruce Adler,1,79293,False,0CKmN3Wwk8W4zjU0pqq2cv,False,Arabian Nights,58,1,track,spotify:track:0CKmN3Wwk8W4zjU0pqq2cv,29EiOQJnxWlX5nVOWQpu3u,66oKiXdIQP7MwN0gPUY0FD
1,Alan Menken,1,85186,False,0dQKabsoT9ojccQgVhLmHN,False,Legend of the Lamp,36,2,track,spotify:track:0dQKabsoT9ojccQgVhLmHN,29EiOQJnxWlX5nVOWQpu3u,5sy77gt4bfsLcSQ8GIe4ZZ
2,Brad Kane,1,142440,False,4wN8Ov3kPZdkJ8XcYxYUGz,False,One Jump Ahead,54,3,track,spotify:track:4wN8Ov3kPZdkJ8XcYxYUGz,29EiOQJnxWlX5nVOWQpu3u,3dAzSJ9lQnJSq5Z0OgDBep
3,Alan Menken,1,112946,False,4Y8ffsPwnVRkFEUqxkQAyz,False,Street Urchins,35,4,track,spotify:track:4Y8ffsPwnVRkFEUqxkQAyz,29EiOQJnxWlX5nVOWQpu3u,5sy77gt4bfsLcSQ8GIe4ZZ
4,Brad Kane,1,61360,False,6LNdgw2RMWQLrlGUzcUO4z,False,One Jump Ahead (Reprise),40,5,track,spotify:track:6LNdgw2RMWQLrlGUzcUO4z,29EiOQJnxWlX5nVOWQpu3u,3dAzSJ9lQnJSq5Z0OgDBep
5,Robin Williams,1,146440,False,70uZXOG2KTAHwcAgp4QyYr,False,Friend Like Me,0,6,track,spotify:track:70uZXOG2KTAHwcAgp4QyYr,29EiOQJnxWlX5nVOWQpu3u,4OrieSJUghNfNx5UuPCgxF
6,Alan Menken,1,99253,False,30J2aUXkTp9vsGWH3tegca,False,To Be Free,37,7,track,spotify:track:30J2aUXkTp9vsGWH3tegca,29EiOQJnxWlX5nVOWQpu3u,5sy77gt4bfsLcSQ8GIe4ZZ
7,Robin Williams,1,171773,False,6q2r0n2ph4w2p7fLbh5TKK,False,Prince Ali,0,8,track,spotify:track:6q2r0n2ph4w2p7fLbh5TKK,29EiOQJnxWlX5nVOWQpu3u,4OrieSJUghNfNx5UuPCgxF
8,Lea Salonga,1,160800,False,5VIfacsWytkcgr7aTt8Tql,False,A Whole New World,51,9,track,spotify:track:5VIfacsWytkcgr7aTt8Tql,29EiOQJnxWlX5nVOWQpu3u,1GlMjIezcLwV3OFlX0uXOv
8,Brad Kane,1,160800,False,5VIfacsWytkcgr7aTt8Tql,False,A Whole New World,51,9,track,spotify:track:5VIfacsWytkcgr7aTt8Tql,29EiOQJnxWlX5nVOWQpu3u,3dAzSJ9lQnJSq5Z0OgDBep


In [88]:
tracks_df.to_csv('../data/tracks.csv')

In [90]:
tracks_df[tracks_df['name'].str.contains('About Bruno')]

,artists,disc_number,duration_ms,explicit,id,is_local,name,popularity,track_number,type,uri,album_id,artists_id
1135,Carolina Gaitán - La Gaita,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,29PgYEggDV3cDP9QYTogwv
1135,Mauro Castillo,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,36CUTsdtNgCwMq6zKD1l8I
1135,Adassa,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,72kqSqk124Vnl1wtT6q9Fh
1135,Rhenzy Feliz,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,2as15AH2BTrPk8v4gyElmr
1135,Diane Guerrero,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,18MFcGBHtyW0mU3ufcm0X1
1135,Stephanie Beatriz,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,5PYToRCsrnvikZg3yl2JMr
1135,Encanto - Cast,1,216120,False,52xJxFP6TqMuO4Yt0eOkMz,False,We Don't Talk About Bruno,74,4,track,spotify:track:52xJxFP6TqMuO4Yt0eOkMz,25L8ck3KGcmCo3901ztPzR,3xLU748QxpTmIVaiNXXg0P
1171,Lin-Manuel Miranda,1,215933,False,7nm7JfzJtp63Y2S8xnru52,False,We Don't Talk About Bruno - Instrumental,44,40,track,spotify:track:7nm7JfzJtp63Y2S8xnru52,25L8ck3KGcmCo3901ztPzR,4aXXDj9aZnlshx7mzj3W1N
